# Join updated NC of CDP and CIP data to microphy NC -  resample cdp with mean

In [2]:
import xarray as xr
import pandas as pd
from datetime import date
import read_flight_report as rfr
import functions

In [3]:
# --- Data input
flight = 'IS22-08'

cdp_file = f'../Results_2022-islas/Processed/CDP_processed/CDP_updated_{flight}.nc'
cip_file = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'

cdp_ds = xr.open_dataset(cdp_file)
cip_ds = xr.open_dataset(cip_file)

In [4]:
print(f'CDP max: {cdp_ds['time'].max().values}, CDP min: {cdp_ds['time'].min().values}')
print(f'CIP max: {cip_ds['time'].max().values}, CIP min: {cip_ds['time'].min().values}')

CDP max: 2022-03-30T17:00:05.828125000, CDP min: 2022-03-30T13:48:34.875000000
CIP max: 2022-03-30T17:00:04.400022000, CIP min: 2022-03-30T13:48:49.400022000


In [7]:
# resample cdp data with mean()
# Test with resampled cdp data

cdp_ds
lwc_da = cdp_ds['LWC corr'].resample(time='5s').mean() # resample

lwc_da

ValueError: Index must be monotonic for resampling

In [5]:
#  Remove milliseconds to ease joining
cdp_ds = functions.floor_to_sec_res(cdp_ds, 'time')
cip_ds = functions.floor_to_sec_res(cip_ds, 'time')
# remove any unwanted duplicate values from cdp
cdp_ds = cdp_ds.sel(time=~cdp_ds.get_index("time").duplicated())

In [6]:
# check for duplicates
#cdp_ds.sel(time=cdp_ds.get_index('time').duplicated())

In [7]:

# merge the two xarrays
microphy_ds = xr.merge([cip_ds, cdp_ds],compat='override',join='left')

# update attrs for variables with parent file
for var_name in cdp_ds.data_vars:
    microphy_ds[var_name].attrs.update({"parent file":cdp_file.split('/')[-1]})
for var_name in cip_ds.data_vars:
    microphy_ds[var_name].attrs.update({"parent file":cip_file.split('/')[-1]})



In [8]:
# remove dataset attributes
microphy_ds =microphy_ds.drop_attrs(deep = False)

# set new dataset attributes
microphy_ds.attrs['safireid']=cip_ds.attrs['safireid']
microphy_ds.attrs['islasid']=cip_ds.attrs['islasid']
microphy_ds.attrs['parent files']=[cip_file.split('/')[-1],cdp_file.split('/')[-1]]
microphy_ds.attrs['date_modified'] = date.today().strftime("%Y-%m-%d")

In [9]:
# calculate Total Water content
microphy_ds['TWC'] = microphy_ds['LWC corr']+microphy_ds['IWC100']
microphy_ds['TWC'].attrs['longname']='Total Water Content'
microphy_ds['TWC'].attrs['description']='TWC calculated from CIP IWC and CDP LWC, assuming all WC from CIP is ice and all WC from CDP is water'
microphy_ds['TWC'].attrs['calculated from']=['LWC corr','IWC100']


# Calculate supercooled liquid fraction for in-cloud values
microphy_ds.attrs['incloud_thres']= 0.01
import numpy as np
microphy_ds['SLF_all'] = microphy_ds['LWC corr']/microphy_ds['TWC']*100
microphy_ds['SLF_all'].attrs['longname']='Supercooled liquid fraction, all points'
microphy_ds['SLF_all'].attrs['unit']='percent'
microphy_ds['SLF_all'].attrs['description']='Supercooled liquid fraction for all observation points, calculated from the Total water content and the Liquid water content'
microphy_ds['SLF_all'].attrs['calculated from']=['LWC corr', 'TWC']

microphy_ds['SLF'] = xr.where(microphy_ds['LWC corr']>microphy_ds.attrs['incloud_thres'],microphy_ds['SLF_all'],np.nan)
microphy_ds['SLF'].attrs['longname']='Supercooled liquid fraction, incloud LWC threshold'
microphy_ds['SLF'].attrs['unit']='percent'
microphy_ds['SLF'].attrs['description']='Supercooled liquid fraction for points where LWC above threshold, calculated from the Total water content and the Liquid water content'
microphy_ds['SLF'].attrs['calculated from']=['LWC corr', 'SLF_all', 'incloud_thres']

microphy_ds['SLF_twc'] = xr.where(microphy_ds['TWC']>microphy_ds.attrs['incloud_thres'],microphy_ds['SLF_all'],np.nan)
microphy_ds['SLF_twc'].attrs['longname']='Supercooled liquid fraction, incloud TWC threshold'
microphy_ds['SLF_twc'].attrs['unit']='percent'
microphy_ds['SLF_twc'].attrs['description']='Supercooled liquid fraction for points where TWC above threshold, calculated from the Total water content and the Liquid water content'
microphy_ds['SLF_twc'].attrs['calculated from']=['SLF_all', 'TWC', 'incloud_thres']

In [10]:
# setting surface conditions
from global_land_mask import globe
sea_ice_file = ''

# determine lan/sea distinction from global_land_mask
condition = globe.is_land(microphy_ds['lat'].values[:, np.newaxis], microphy_ds['lon'].values)
surface_cond = xr.where(condition, 'land', 'sea')
microphy_ds['surface_cond']= (('lat','lon'), surface_cond)

# set attributes for the surface conditions
microphy_ds['surface_cond'].attrs['longname']='Surface conditions'
microphy_ds['surface_cond'].attrs['standard_name']='area_type'
microphy_ds['surface_cond'].attrs['description']= 'Nature of surface below measurements. Land and sea is determined by global_land_mask package, sea ice from satellite observations'
microphy_ds['surface_cond'].attrs['sea_ice_file']= sea_ice_file


In [11]:
# Get sea ice concentation
#date = str(microphy_ds.time[0].values.astype('datetime64[D]')).replace('-', '')
#date

#sic_ds = xr.open_dataset('sea_ice_satellite/asi-n6250-' + str(date) + '-5.4_regridded.nc')
#sic_ds.close()

# rename data variable and update attributes
#sic_ds['sic'] = sic_ds['__xarray_dataarray_variable__'].assign_attrs(units="Percent", description="Sea Ice Concentration")
#sic_ds = sic_ds.drop_vars(['__xarray_dataarray_variable__'])

#test = sic_ds['sic'].sel(lat=('lat', microphy_ds.lat),lon=('lon',microphy_ds.lon), method='nearest')
#test
#microphy_ds['sic']=sic_ds.sel(lat=microphy_ds.lat, lon=microphy_ds.lon, method='nearest')

In [12]:
# write to netcdf file
microphy_ds.to_netcdf(path=f'../Results_2022-islas/Processed/ISLAS_processed/microphy_5s_{flight}.nc', mode='w')
cip_ds.close
cdp_ds.close

<bound method DataWithCoords.close of <xarray.Dataset> Size: 6MB
Dimensions:                 (time: 11461, CDP_Bin: 30)
Coordinates:
    lat                     (time) float32 46kB ...
    lon                     (time) float32 46kB ...
    alt                     (time) float32 46kB ...
  * CDP_Bin                 (CDP_Bin) int64 240B 1 2 3 4 5 6 ... 26 27 28 29 30
  * time                    (time) datetime64[ns] 92kB 2022-03-30T13:58:54 .....
Data variables: (12/34)
    End Seconds             (time) float64 92kB ...
    Day of Year             (time) float64 92kB ...
    Year                    (time) float64 92kB ...
    Status                  (time) float64 92kB ...
    DOF Reject Counts       (time) float64 92kB ...
    Avg Transit Reject      (time) float64 92kB ...
    ...                      ...
    CDP Bin Particle Count  (time, CDP_Bin) float64 3MB ...
    TAS reduce              (time) float32 46kB ...
    TAS correction factor   (time) float64 92kB ...
    Number Conc c